In [4]:
import torch
import time
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer,Pipeline 
from transformers.generation.utils import GenerationConfig

In [2]:
def init_model(badwords = []):
    model = AutoModelForCausalLM.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        torch_dtype=torch.float16,
        device_map="cuda:0",
        trust_remote_code=True,
    )
    model.generation_config = GenerationConfig.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        temperature=0.8,
        no_repeat_ngram_size=8,
        bad_words_ids = badwords
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        use_fast=False,
        trust_remote_code=True
    )
    print(model.generation_config)
    return model, tokenizer

tokenizer = AutoTokenizer.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        use_fast=False,
        trust_remote_code=True
    )

In [3]:
connective_words = ['首先', '其次', '然后', '接着', '最后', '总之','总的来说','总而言之','一方面','另一方面']
badwords = tokenizer(connective_words, add_special_tokens=False).input_ids
model,tokenizer = init_model(badwords=badwords)
print(model.generation_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GenerationConfig {
  "assistant_token_id": 196,
  "bad_words_ids": [
    [
      4547
    ],
    [
      10108
    ],
    [
      3258
    ],
    [
      9969
    ],
    [
      2534
    ],
    [
      14748
    ],
    [
      37425
    ],
    [
      50515
    ],
    [
      10839
    ],
    [
      11395
    ]
  ],
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 2048,
  "no_repeat_ngram_size": 8,
  "pad_token_id": 0,
  "repetition_penalty": 1.05,
  "temperature": 0.8,
  "top_k": 5,
  "top_p": 0.85,
  "transformers_version": "4.33.2",
  "user_token_id": 195
}

GenerationConfig {
  "assistant_token_id": 196,
  "bad_words_ids": [
    [
      4547
    ],
    [
      10108
    ],
    [
      3258
    ],
    [
      9969
    ],
    [
      2534
    ],
    [
      14748
    ],
    [
      37425
    ],
    [
      50515
    ],
    [
      10839
    ],
    [
      11395
    ]
  ],
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_toke

In [12]:
prompt =  """
你是一名小红书上的 美妆 博主，请你制作一篇 明星仿妆 的教程分享。

教程步骤： 至少5个步骤 

产品清单： 列出所使用的主要产品 

平台文风： 小红书 。

文案要求： 逻辑清晰，可以适当加一些表情符号。
"""
messages = []
messages.append({"role": "user", "content": prompt})
response = model.chat(tokenizer, messages)
print(response)

🌟 明星仿妆教程：模仿赵丽颖的甜美妆容 🌟

🎉 大家好！我是你们的美妆博主小美，今天我将教大家如何模仿赵丽颖的甜美妆容。接下来让我们开始吧！

1️⃣ 底妆：第一步是打造完美的底妆。我们需要选择一款适合我们的粉底液，我用的是雅诗兰黛的双头粉底液，它可以打造出自然的肤色。用化妆海绵均匀涂抹在脸上，遮盖瑕疵和均匀肤色。

2️⃣ 遮瑕：接下来我们要用遮瑕膏遮盖黑眼圈和痘印。我用的是NARS的遮瑕膏，它非常适合遮盖瑕疵。用遮瑕刷蘸取适量遮瑕膏，点拍在需要遮盖的地方，然后用手指轻轻拍打，让遮瑕膏更好地融入皮肤。

3️⃣ 眼妆：接下来我们开始画眼妆。先用眼影刷蘸取一些大地色眼影，打底整个眼皮。然后用粉色眼影在眼皮中央晕染，增加眼部立体感。再用亮片眼影在眼皮中央和眼头处点缀，增加眼部闪亮感。最后是画眼线，我选择的是棕色的眼线笔，让它更自然。

4️⃣ 睫毛：为了让眼睛更有神，我们需要贴双眼皮贴和刷睫毛。我先贴上双眼皮贴，然后用睫毛夹夹翘睫毛，最后再刷上浓密的睫毛膏。

5️⃣ 腮红：为了让脸部更有气色，我们需要打腮红。我用的是NARS的腮红，颜色非常自然。将腮红从颧骨处向太阳穴方向轻轻扫去，看到没有？脸部立刻有了红润的感觉！

6️⃣ 唇妆：最后是唇妆部分，我用的是YSL的水光唇釉，颜色非常甜美。先将唇部保湿做好，然后将唇釉均匀涂抹在唇部，最后再轻轻抿一下，让唇釉更加服帖。

💖 现在，一个甜美的赵丽颖仿妆就完成啦！希望大家喜欢这个教程，也欢迎大家私信我交流美妆心得哦~爱你们！


In [7]:
df = pd.read_excel("NewPrompt.xlsx",header=None)
df = df.assign(round0_ans = None , round1_ans = None)
str(df.iloc[0,1]) == "nan"

True

In [ ]:
for i in range(len(df)):
    messages = []
    if str(df.iloc[i,1])  == "nan":
        messages.append({"role": "user", "content": df.iloc[i,0]})
        response = model.chat(tokenizer, messages)
        df.iloc[i,2] = response
    else:
        messages = [history]
        messages.append({"role": "user", "content": df.iloc[i,1]})
        response,history = model.chat(tokenizer, messages)
        df.iloc[i,3] = response
df.to_excel("NewPromptEval.xlsx")

NameError: name 'prompt_list' is not defined